In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/swcon_capstone

/content/drive/My Drive/Colab Notebooks/swcon_capstone


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
X_train_0703_1 = np.load("X_train_0703_1.npy")
y_train_0703_1 = np.load("y_train_0703_1.npy")
X_test_0703_1 = np.load("X_test_0703_1.npy")
y_test_0703_1 = np.load("y_test_0703_1.npy")

X_train_0703_2 = np.load("X_train_0703_2.npy")
y_train_0703_2 = np.load("y_train_0703_2.npy")
X_test_0703_2 = np.load("X_test_0703_2.npy")
y_test_0703_2 = np.load("y_test_0703_2.npy")

X_train_0705 = np.load("X_train_0705.npy")
y_train_0705 = np.load("y_train_0705.npy")
X_test_0705 = np.load("X_test_0705.npy")
y_test_0705 = np.load("y_test_0705.npy")

X_train_0707 = np.load("X_train_0707.npy")
y_train_0707 = np.load("y_train_0707.npy")
X_test_0707 = np.load("X_test_0707.npy")
y_test_0707 = np.load("y_test_0707.npy")

X_train_0710_1 = np.load("X_train_0710_1.npy")
y_train_0710_1 = np.load("y_train_0710_1.npy")
X_test_0710_1 = np.load("X_test_0710_1.npy")
y_test_0710_1 = np.load("y_test_0710_1.npy")

X_train_0710_2 = np.load("X_train_0710_2.npy")
y_train_0710_2 = np.load("y_train_0710_2.npy")
X_test_0710_2 = np.load("X_test_0710_2.npy")
y_test_0710_2 = np.load("y_test_0710_2.npy")

X_train_0710_3 = np.load("X_train_0710_3.npy")
y_train_0710_3 = np.load("y_train_0710_3.npy")
X_test_0710_3 = np.load("X_test_0710_3.npy")
y_test_0710_3 = np.load("y_test_0710_3.npy")

X_train_0712 = np.load("X_train_0712.npy")
y_train_0712 = np.load("y_train_0712.npy")
X_test_0712 = np.load("X_test_0712.npy")
y_test_0712 = np.load("y_test_0712.npy")

X_train_0713 = np.load("X_train_0713.npy")
y_train_0713 = np.load("y_train_0713.npy")
X_test_0713 = np.load("X_test_0713.npy")
y_test_0713 = np.load("y_test_0713.npy")

In [ ]:
X_train = np.concatenate((X_train_0703_1, X_train_0703_2, X_train_0705, X_train_0707, X_train_0710_1, X_train_0710_2, X_train_0710_3, X_train_0712, X_train_0713), axis=0)
y_train = np.concatenate((y_train_0703_1, y_train_0703_2, y_train_0705, y_train_0707, y_train_0710_1, y_train_0710_2, y_train_0710_3, y_train_0712, y_train_0713), axis=0)
X_test = np.concatenate((X_test_0703_1, X_test_0703_2, X_test_0705, X_test_0707, X_test_0710_1, X_test_0710_2, X_test_0710_3, X_test_0712, X_test_0713), axis=0)
y_test = np.concatenate((y_test_0703_1, y_test_0703_2, y_test_0705, y_test_0707, y_test_0710_1, y_test_0710_2, y_test_0710_3, y_test_0712, y_test_0713), axis=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(157501, 160)
(39381, 160)


### 아래는  전 데이터셋에 대해 머러 모델을 돌린 것

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 512
num_epochs = len(y_train) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=512, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/307], Step [100/308], Loss: 0.3804796636104584
Epoch [1/307], Step [200/308], Loss: 0.34118932485580444
Epoch [1/307], Step [300/308], Loss: 0.2741008400917053
Epoch [2/307], Step [100/308], Loss: 0.2053924947977066
Epoch [2/307], Step [200/308], Loss: 0.16862598061561584
Epoch [2/307], Step [300/308], Loss: 0.1630077064037323
Epoch [3/307], Step [100/308], Loss: 0.14090394973754883
Epoch [3/307], Step [200/308], Loss: 0.12961216270923615
Epoch [3/307], Step [300/308], Loss: 0.10931194573640823
Epoch [4/307], Step [100/308], Loss: 0.08132340759038925
Epoch [4/307], Step [200/308], Loss: 0.07448147237300873
Epoch [4/307], Step [300/308], Loss: 0.06636938452720642
Epoch [5/307], Step [100/308], Loss: 0.08204329758882523
Epoch [5/307], Step [200/308], Loss: 0.06890326738357544
Epoch [5/307], Step [300/308], Loss: 0.04920678585767746
Epoch [6/307], Step [100/308], Loss: 0.0635007843375206
Epoch [6/307], Step [200/308], Loss: 0.04735895246267319
Epoch [6/307], Step [300/308], Loss:

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (data, labels) in enumerate(test_loader):
        data = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 100.0
Iteration:2,  Accuracy: 100.0
Iteration:3,  Accuracy: 100.0
Iteration:4,  Accuracy: 100.0
Iteration:5,  Accuracy: 100.0
Iteration:6,  Accuracy: 100.0
Iteration:7,  Accuracy: 100.0
Iteration:8,  Accuracy: 99.66666666666667
Iteration:9,  Accuracy: 99.7
Iteration:10,  Accuracy: 99.72727272727273
Iteration:11,  Accuracy: 99.66666666666667
Iteration:12,  Accuracy: 99.6923076923077
Iteration:13,  Accuracy: 99.71428571428571
Iteration:14,  Accuracy: 99.73333333333333
Iteration:15,  Accuracy: 99.75
Iteration:16,  Accuracy: 99.6470588235294
Iteration:17,  Accuracy: 99.66666666666667
Iteration:18,  Accuracy: 99.6842105263158
Iteration:19,  Accuracy: 99.7
Iteration:20,  Accuracy: 99.71428571428571
Iteration:21,  Accuracy: 99.72727272727273
Iteration:22,  Accuracy: 99.73913043478261
Iteration:23,  Accuracy: 99.70833333333333
Iteration:24,  Accuracy: 99.72
Iteration:25,  Accuracy: 99.73076923076923
Iteration:26,  Accuracy: 99.7407407407407

### 20개 아니고 10개씩 잘라서 진행

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 400
num_epochs = len(y_train[100000:]) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train[100000:], y_train[100000:]) # 400,160
train_loader = DataLoader(dataset=train_dataset, batch_size=400, shuffle=True)

# Train the model 400*20*8 = 800*10*8
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = images[:, 10:]
        #print(images.shape)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/143], Step [100/144], Loss: 0.27681219577789307
Epoch [2/143], Step [100/144], Loss: 0.21070636808872223
Epoch [3/143], Step [100/144], Loss: 0.20594066381454468
Epoch [4/143], Step [100/144], Loss: 0.18833181262016296
Epoch [5/143], Step [100/144], Loss: 0.1516430526971817
Epoch [6/143], Step [100/144], Loss: 0.10275159776210785
Epoch [7/143], Step [100/144], Loss: 0.09995552897453308
Epoch [8/143], Step [100/144], Loss: 0.11717662215232849
Epoch [9/143], Step [100/144], Loss: 0.07111965864896774
Epoch [10/143], Step [100/144], Loss: 0.04940161108970642
Epoch [11/143], Step [100/144], Loss: 0.06520748138427734
Epoch [12/143], Step [100/144], Loss: 0.02650388516485691
Epoch [13/143], Step [100/144], Loss: 0.043343354016542435
Epoch [14/143], Step [100/144], Loss: 0.042931489646434784
Epoch [15/143], Step [100/144], Loss: 0.02375202812254429
Epoch [16/143], Step [100/144], Loss: 0.031941719353199005
Epoch [17/143], Step [100/144], Loss: 0.026157855987548828
Epoch [18/143], Step

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test[30000:], y_test[30000:])
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = images[:, 10:]
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 100.0
Iteration:2,  Accuracy: 99.66666666666667
Iteration:3,  Accuracy: 99.5
Iteration:4,  Accuracy: 99.4
Iteration:5,  Accuracy: 99.33333333333333
Iteration:6,  Accuracy: 99.28571428571429
Iteration:7,  Accuracy: 99.125
Iteration:8,  Accuracy: 99.0
Iteration:9,  Accuracy: 99.1
Iteration:10,  Accuracy: 99.18181818181819
Iteration:11,  Accuracy: 99.25
Iteration:12,  Accuracy: 99.3076923076923
Iteration:13,  Accuracy: 99.14285714285714
Iteration:14,  Accuracy: 99.2
Iteration:15,  Accuracy: 99.25
Iteration:16,  Accuracy: 99.29411764705883
Iteration:17,  Accuracy: 99.27777777777777
Iteration:18,  Accuracy: 99.3157894736842
Iteration:19,  Accuracy: 99.35
Iteration:20,  Accuracy: 99.38095238095238
Iteration:21,  Accuracy: 99.36363636363636
Iteration:22,  Accuracy: 99.3913043478261
Iteration:23,  Accuracy: 99.41666666666667
Iteration:24,  Accuracy: 99.36
Iteration:25,  Accuracy: 99.3076923076923
Iteration:26,  Accuracy: 99.29629629629629
I

# Multi Class Classification


In [8]:
X_train_0721 = np.load("X_train_0721.npy")
y_train_0721 = np.load("y_train_0721.npy")
X_test_0721 = np.load("X_test_0721.npy")
y_test_0721 = np.load("y_test_0721.npy")

X_train_0723 = np.load("X_train_0723.npy")
y_train_0723 = np.load("y_train_0723.npy")
X_test_0723 = np.load("X_test_0723.npy")
y_test_0723 = np.load("y_test_0723.npy")

In [ ]:
X_test_0721.shape

(5965, 160)

In [7]:
X_train = np.concatenate(( X_train_0721, X_train_0723), axis=0)
y_train = np.concatenate(( y_train_0721, y_train_0723), axis=0)
X_test = np.concatenate(( X_test_0721, X_test_0723), axis=0)
y_test = np.concatenate((y_test_0721, y_test_0723), axis=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(49669, 160)
(12419, 160)


In [ ]:
print(np.bincount(y_test_0721))

[2944  485  295  299  299  311  306  323  703]


In [ ]:
print(np.bincount(y_train))
print(np.bincount(y_test))

[25324  3960  2326  2359  2379  2449  2464  2589  5819]
[6331  991  581  591  595  612  616  647 1455]


### 아래는 20개짜리

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 9
batch_size = 512
num_epochs = len(y_train) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=512, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/331], Step [100/332], Loss: 0.561526894569397
Epoch [1/331], Step [200/332], Loss: 0.41244256496429443
Epoch [1/331], Step [300/332], Loss: 0.5402791500091553
Epoch [2/331], Step [100/332], Loss: 0.3541595935821533
Epoch [2/331], Step [200/332], Loss: 0.3199812173843384
Epoch [2/331], Step [300/332], Loss: 0.3521575927734375
Epoch [3/331], Step [100/332], Loss: 0.18315131962299347
Epoch [3/331], Step [200/332], Loss: 0.16422076523303986
Epoch [3/331], Step [300/332], Loss: 0.17044837772846222
Epoch [4/331], Step [100/332], Loss: 0.0695042535662651
Epoch [4/331], Step [200/332], Loss: 0.20512594282627106
Epoch [4/331], Step [300/332], Loss: 0.07928928732872009
Epoch [5/331], Step [100/332], Loss: 0.03247762471437454
Epoch [5/331], Step [200/332], Loss: 0.05850009247660637
Epoch [5/331], Step [300/332], Loss: 0.04632242023944855
Epoch [6/331], Step [100/332], Loss: 0.04940245673060417
Epoch [6/331], Step [200/332], Loss: 0.05454302579164505
Epoch [6/331], Step [300/332], Loss: 0

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device) # 이런게 실제 동작 부위보다 시간이 오래걸림
        outputs = model(images) # 연산 시간이라 볼수도있는데
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 98.4375
Iteration:2,  Accuracy: 98.95833333333333
Iteration:3,  Accuracy: 98.4375
Iteration:4,  Accuracy: 98.75
Iteration:5,  Accuracy: 98.4375
Iteration:6,  Accuracy: 98.21428571428571
Iteration:7,  Accuracy: 98.4375
Iteration:8,  Accuracy: 98.61111111111111
Iteration:9,  Accuracy: 98.75
Iteration:10,  Accuracy: 98.86363636363636
Iteration:11,  Accuracy: 98.69791666666667
Iteration:12,  Accuracy: 98.5576923076923
Iteration:13,  Accuracy: 98.66071428571429
Iteration:14,  Accuracy: 98.54166666666667
Iteration:15,  Accuracy: 98.4375
Iteration:16,  Accuracy: 98.52941176470588
Iteration:17,  Accuracy: 98.61111111111111
Iteration:18,  Accuracy: 98.6842105263158
Iteration:19,  Accuracy: 98.75
Iteration:20,  Accuracy: 98.66071428571429
Iteration:21,  Accuracy: 98.7215909090909
Iteration:22,  Accuracy: 98.77717391304348
Iteration:23,  Accuracy: 98.828125
Iteration:24,  Accuracy: 98.875
Iteration:25,  Accuracy: 98.91826923076923
Iteration:26

### 10개짜리

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 9
batch_size = 150
num_epochs = len(y_train[25000:]) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train[25000:], y_train[25000:]) # 400,160
train_loader = DataLoader(dataset=train_dataset, batch_size=150, shuffle=True)

# Train the model 400*20*8 = 800*10*8
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = images[:, 10:]
        #print(images.shape)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/164], Step [100/165], Loss: 0.7253389954566956
Epoch [2/164], Step [100/165], Loss: 0.458177387714386
Epoch [3/164], Step [100/165], Loss: 0.27630671858787537
Epoch [4/164], Step [100/165], Loss: 0.3011864125728607
Epoch [5/164], Step [100/165], Loss: 0.24990302324295044
Epoch [6/164], Step [100/165], Loss: 0.15201915800571442
Epoch [7/164], Step [100/165], Loss: 0.13964848220348358
Epoch [8/164], Step [100/165], Loss: 0.16478371620178223
Epoch [9/164], Step [100/165], Loss: 0.12459197640419006
Epoch [10/164], Step [100/165], Loss: 0.07041636109352112
Epoch [11/164], Step [100/165], Loss: 0.06382420659065247
Epoch [12/164], Step [100/165], Loss: 0.04761909320950508
Epoch [13/164], Step [100/165], Loss: 0.07167492061853409
Epoch [14/164], Step [100/165], Loss: 0.020317669957876205
Epoch [15/164], Step [100/165], Loss: 0.020299185067415237
Epoch [16/164], Step [100/165], Loss: 0.06359891593456268
Epoch [17/164], Step [100/165], Loss: 0.01221522968262434
Epoch [18/164], Step [100

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test[6000:], y_test[6000:])
test_loader = DataLoader(dataset=test_dataset, batch_size=60, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        images = images[:, 10:]
        labels = labels.to(device) # 이런게 실제 동작 부위보다 시간이 오래걸림
        outputs = model(images) # 연산 시간이라 볼수도있는데
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 96.66666666666667
Iteration:1,  Accuracy: 96.66666666666667
Iteration:2,  Accuracy: 97.22222222222223
Iteration:3,  Accuracy: 97.91666666666667
Iteration:4,  Accuracy: 98.33333333333333
Iteration:5,  Accuracy: 98.33333333333333
Iteration:6,  Accuracy: 98.57142857142857
Iteration:7,  Accuracy: 98.33333333333333
Iteration:8,  Accuracy: 98.51851851851852
Iteration:9,  Accuracy: 98.5
Iteration:10,  Accuracy: 98.18181818181819
Iteration:11,  Accuracy: 98.33333333333333
Iteration:12,  Accuracy: 98.2051282051282
Iteration:13,  Accuracy: 97.97619047619048
Iteration:14,  Accuracy: 98.11111111111111
Iteration:15,  Accuracy: 98.02083333333333
Iteration:16,  Accuracy: 98.13725490196079
Iteration:17,  Accuracy: 98.05555555555556
Iteration:18,  Accuracy: 97.98245614035088
Iteration:19,  Accuracy: 97.83333333333333
Iteration:20,  Accuracy: 97.93650793650794
Iteration:21,  Accuracy: 97.95454545454545
Iteration:22,  Accuracy: 97.89855072463769
Iteration:23,  Accuracy: 97.9166666

# 논문과 직접적 비교를 위한 학습
## Binary Classification

### 1. 2020-07-12 데이터로만 학습 및 테스트
##### 논문에서의 결과) 99%

In [ ]:
X_train_0712 = np.load("X_train_0712.npy")
y_train_0712 = np.load("y_train_0712.npy")
X_test_0712 = np.load("X_test_0712.npy")
y_test_0712 = np.load("y_test_0712.npy")

In [ ]:
print(X_train_0712.shape)
print(X_test_0712.shape)
print(np.bincount(y_train_0712))
print(np.bincount(y_test_0712))

(7193, 160)
(1799, 160)
[3371 3822]
[843 956]


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 32
num_epochs = len(y_train_0712) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_0712, y_train_0712)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/224], Step [100/225], Loss: 0.16171139478683472
Epoch [1/224], Step [200/225], Loss: 0.11455738544464111
Epoch [2/224], Step [100/225], Loss: 0.058471713215112686
Epoch [2/224], Step [200/225], Loss: 0.018002161756157875
Epoch [3/224], Step [100/225], Loss: 0.2823757827281952
Epoch [3/224], Step [200/225], Loss: 0.08304735273122787
Epoch [4/224], Step [100/225], Loss: 0.0025273743085563183
Epoch [4/224], Step [200/225], Loss: 0.004826669581234455
Epoch [5/224], Step [100/225], Loss: 0.0034316948149353266
Epoch [5/224], Step [200/225], Loss: 0.024067139253020287
Epoch [6/224], Step [100/225], Loss: 0.009466120973229408
Epoch [6/224], Step [200/225], Loss: 0.0032103373669087887
Epoch [7/224], Step [100/225], Loss: 0.18426887691020966
Epoch [7/224], Step [200/225], Loss: 0.0359390527009964
Epoch [8/224], Step [100/225], Loss: 0.15726642310619354
Epoch [8/224], Step [200/225], Loss: 0.0031849767547100782
Epoch [9/224], Step [100/225], Loss: 0.001125242910347879
Epoch [9/224], Step

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_0712, y_test_0712)
test_loader = DataLoader(dataset=test_dataset, batch_size=20, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre_0712 = predicted.tolist()
        else:
          y_pre_0712.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names =  ['Nominal', 'Faulty']
print(classification_report(y_test_0712.tolist(), y_pre_0712, target_names=target_names))

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 100.0
Iteration:2,  Accuracy: 100.0
Iteration:3,  Accuracy: 100.0
Iteration:4,  Accuracy: 100.0
Iteration:5,  Accuracy: 100.0
Iteration:6,  Accuracy: 100.0
Iteration:7,  Accuracy: 100.0
Iteration:8,  Accuracy: 100.0
Iteration:9,  Accuracy: 100.0
Iteration:10,  Accuracy: 100.0
Iteration:11,  Accuracy: 100.0
Iteration:12,  Accuracy: 100.0
Iteration:13,  Accuracy: 100.0
Iteration:14,  Accuracy: 100.0
Iteration:15,  Accuracy: 100.0
Iteration:16,  Accuracy: 100.0
Iteration:17,  Accuracy: 100.0
Iteration:18,  Accuracy: 100.0
Iteration:19,  Accuracy: 100.0
Iteration:20,  Accuracy: 100.0
Iteration:21,  Accuracy: 100.0
Iteration:22,  Accuracy: 100.0
Iteration:23,  Accuracy: 100.0
Iteration:24,  Accuracy: 100.0
Iteration:25,  Accuracy: 100.0
Iteration:26,  Accuracy: 100.0
Iteration:27,  Accuracy: 100.0
Iteration:28,  Accuracy: 100.0
Iteration:29,  Accuracy: 100.0
Iteration:30,  Accuracy: 100.0
Iteration:31,  Accuracy: 100.0
Iteration:32,  Acc

### 12일 데이터를 학습한 모델로 13일 데이터 분류하기
##### 논문 결과) acc 61%

In [ ]:
X_0713_transformed = np.load("X_0713_transformed.npy")
y_0713_transformed = np.load("y_0713_transformed.npy")

In [ ]:
X_train = np.concatenate(( X_train_0712, X_test_0712), axis=0)
y_train = np.concatenate(( y_train_0712, y_test_0712), axis=0)

In [ ]:
print(X_train.shape)
print(np.bincount(y_train))

(8992, 160)
[4214 4778]


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 35
num_epochs = len(y_train) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=35, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/256], Step [100/257], Loss: 0.16657941043376923
Epoch [1/256], Step [200/257], Loss: 0.06347063928842545
Epoch [2/256], Step [100/257], Loss: 0.021714773029088974
Epoch [2/256], Step [200/257], Loss: 0.006323862820863724
Epoch [3/256], Step [100/257], Loss: 0.10770280659198761
Epoch [3/256], Step [200/257], Loss: 0.026077792048454285
Epoch [4/256], Step [100/257], Loss: 0.029044324532151222
Epoch [4/256], Step [200/257], Loss: 0.04301884025335312
Epoch [5/256], Step [100/257], Loss: 0.12814775109291077
Epoch [5/256], Step [200/257], Loss: 0.0026919811498373747
Epoch [6/256], Step [100/257], Loss: 0.010787892155349255
Epoch [6/256], Step [200/257], Loss: 0.004405262414366007
Epoch [7/256], Step [100/257], Loss: 0.06666428595781326
Epoch [7/256], Step [200/257], Loss: 0.0036222219932824373
Epoch [8/256], Step [100/257], Loss: 0.005862861406058073
Epoch [8/256], Step [200/257], Loss: 0.12053440511226654
Epoch [9/256], Step [100/257], Loss: 0.035588521510362625
Epoch [9/256], Step

In [ ]:
# Test the model
test_dataset = CustomDataset(X_0713_transformed, y_0713_transformed)
test_loader = DataLoader(dataset=test_dataset, batch_size=5, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre = predicted.tolist()
        else:
          y_pre.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names =  ['Nominal', 'Faulty']
print(classification_report(y_0713_transformed.tolist(), y_pre, target_names=target_names))

Iteration:0,  Accuracy: 80.0
Iteration:1,  Accuracy: 90.0
Iteration:2,  Accuracy: 93.33333333333333
Iteration:3,  Accuracy: 95.0
Iteration:4,  Accuracy: 92.0
Iteration:5,  Accuracy: 86.66666666666667
Iteration:6,  Accuracy: 88.57142857142857
Iteration:7,  Accuracy: 90.0
Iteration:8,  Accuracy: 91.11111111111111
Iteration:9,  Accuracy: 92.0
Iteration:10,  Accuracy: 92.72727272727273
Iteration:11,  Accuracy: 91.66666666666667
Iteration:12,  Accuracy: 92.3076923076923
Iteration:13,  Accuracy: 91.42857142857143
Iteration:14,  Accuracy: 90.66666666666667
Iteration:15,  Accuracy: 91.25
Iteration:16,  Accuracy: 91.76470588235294
Iteration:17,  Accuracy: 90.0
Iteration:18,  Accuracy: 88.42105263157895
Iteration:19,  Accuracy: 89.0
Iteration:20,  Accuracy: 88.57142857142857
Iteration:21,  Accuracy: 88.18181818181819
Iteration:22,  Accuracy: 88.69565217391305
Iteration:23,  Accuracy: 89.16666666666667
Iteration:24,  Accuracy: 88.8
Iteration:25,  Accuracy: 89.23076923076923
Iteration:26,  Accurac

### 3. 2020-07-13 데이터로만 학습 및 테스트<br>
##### 논문 결과) Acc 97%

In [ ]:
print(y_train_0713.shape)
print(y_test_0713.shape)

(7189,)
(1798,)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 16
num_epochs = len(y_test_0713) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_0713, y_train_0713)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/112], Step [100/450], Loss: 0.419612318277359
Epoch [1/112], Step [200/450], Loss: 0.02647334896028042
Epoch [1/112], Step [300/450], Loss: 0.06020445004105568
Epoch [1/112], Step [400/450], Loss: 0.26374879479408264
Epoch [2/112], Step [100/450], Loss: 0.12959034740924835
Epoch [2/112], Step [200/450], Loss: 0.015097755007445812
Epoch [2/112], Step [300/450], Loss: 0.1302727609872818
Epoch [2/112], Step [400/450], Loss: 0.18510979413986206
Epoch [3/112], Step [100/450], Loss: 0.3548528850078583
Epoch [3/112], Step [200/450], Loss: 0.23519019782543182
Epoch [3/112], Step [300/450], Loss: 0.2643558084964752
Epoch [3/112], Step [400/450], Loss: 0.08168578147888184
Epoch [4/112], Step [100/450], Loss: 0.16218599677085876
Epoch [4/112], Step [200/450], Loss: 0.018866531550884247
Epoch [4/112], Step [300/450], Loss: 0.007489549461752176
Epoch [4/112], Step [400/450], Loss: 0.09179055690765381
Epoch [5/112], Step [100/450], Loss: 0.02490401640534401
Epoch [5/112], Step [200/450], Lo

## Output 연산 소요 시간 측정까지 겸함

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_0713, y_test_0713)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False) #batch size 원래 10이었음

gpu_time_list=[]
gpu_time_list = np.array(gpu_time_list)
output_time_list=[]
output_time_list = np.array(output_time_list)
all_time_list=[]
all_time_list = np.array(all_time_list)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (data, labels) in enumerate(test_loader):
        time1 = time.time()
        data = data.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        time2 = time.time()
        outputs = model(images)
        time3 = time.time()
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre_0713 = predicted.tolist()
        else:
          y_pre_0713.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))
        gpu_time_list = np.append(gpu_time_list, time2-time1)
        output_time_list = np.append(output_time_list, time3-time2)
        all_time_list = np.append(all_time_list, time3-time1)

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))
    print("GPU에 올리는데 걸린 평균 시간:{}, model에 넣고 output 계산하는데 걸린 평균 시간:{}, count 제외 output 나오기까지 총 소요 시간 평균:{}".format(gpu_time_list.mean(), output_time_list.mean(), all_time_list.mean()))
    print("GPU에 올리는데 걸린 max 시간:{}, model에 넣고 output 계산하는데 걸린 max 시간:{}, count 제외 output 나오기까지 총 소요 시간 max:{}".format(gpu_time_list.max(), output_time_list.max(), all_time_list.max()))
    print("GPU에 올리는데 걸린 min 시간:{}, model에 넣고 output 계산하는데 걸린 min 시간:{}, count 제외 output 나오기까지 총 소요 시간 min:{}".format(gpu_time_list.min(), output_time_list.min(), all_time_list.min()))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names =  ['Nominal', 'Faulty']
print(classification_report(y_test_0713.tolist(), y_pre_0713, target_names=target_names))

Iteration:0,  Accuracy: 80.0
Iteration:1,  Accuracy: 90.0
Iteration:2,  Accuracy: 93.33333333333333
Iteration:3,  Accuracy: 95.0
Iteration:4,  Accuracy: 96.0
Iteration:5,  Accuracy: 96.66666666666667
Iteration:6,  Accuracy: 97.14285714285714
Iteration:7,  Accuracy: 97.5
Iteration:8,  Accuracy: 96.66666666666667
Iteration:9,  Accuracy: 97.0
Iteration:10,  Accuracy: 97.27272727272727
Iteration:11,  Accuracy: 97.5
Iteration:12,  Accuracy: 97.6923076923077
Iteration:13,  Accuracy: 97.85714285714286
Iteration:14,  Accuracy: 98.0
Iteration:15,  Accuracy: 98.125
Iteration:16,  Accuracy: 97.6470588235294
Iteration:17,  Accuracy: 97.77777777777777
Iteration:18,  Accuracy: 97.89473684210526
Iteration:19,  Accuracy: 98.0
Iteration:20,  Accuracy: 98.0952380952381
Iteration:21,  Accuracy: 98.18181818181819
Iteration:22,  Accuracy: 98.26086956521739
Iteration:23,  Accuracy: 98.33333333333333
Iteration:24,  Accuracy: 98.4
Iteration:25,  Accuracy: 98.46153846153847
Iteration:26,  Accuracy: 98.51851851

### 4. 13일 데이터를 학습한 모델로 12일 데이터 분류하기
##### 논문 결과) acc 87%

In [ ]:
X_train_0713 = np.load("X_train_0713.npy")
y_train_0713 = np.load("y_train_0713.npy")
X_test_0713 = np.load("X_test_0713.npy")
y_test_0713 = np.load("y_test_0713.npy")

In [ ]:
X_train = np.concatenate(( X_train_0713, X_test_0713), axis=0)
y_train = np.concatenate((y_train_0713, y_test_0713), axis=0)

In [ ]:
print(X_train.shape)
print(np.bincount(y_train))

(8987, 160)
[5842 3145]


In [ ]:
X_0712_transformed = np.load("X_0712_transformed.npy")
y_0712_transformed = np.load("y_0712_transformed.npy")

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 40
num_epochs = len(y_train) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=40, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/224], Step [100/225], Loss: 0.19490346312522888
Epoch [1/224], Step [200/225], Loss: 0.10253246128559113
Epoch [2/224], Step [100/225], Loss: 0.028521766886115074
Epoch [2/224], Step [200/225], Loss: 0.015614578500390053
Epoch [3/224], Step [100/225], Loss: 0.14765718579292297
Epoch [3/224], Step [200/225], Loss: 0.002417404670268297
Epoch [4/224], Step [100/225], Loss: 0.011181195266544819
Epoch [4/224], Step [200/225], Loss: 0.016713330522179604
Epoch [5/224], Step [100/225], Loss: 0.002504515927284956
Epoch [5/224], Step [200/225], Loss: 0.004899428226053715
Epoch [6/224], Step [100/225], Loss: 0.002659937832504511
Epoch [6/224], Step [200/225], Loss: 0.002493372419849038
Epoch [7/224], Step [100/225], Loss: 0.006979700177907944
Epoch [7/224], Step [200/225], Loss: 0.10469750314950943
Epoch [8/224], Step [100/225], Loss: 0.028265338391065598
Epoch [8/224], Step [200/225], Loss: 0.005444645881652832
Epoch [9/224], Step [100/225], Loss: 0.029413830488920212
Epoch [9/224], Ste

In [ ]:
# Test the model
test_dataset = CustomDataset(X_0712_transformed, y_0712_transformed)
test_loader = DataLoader(dataset=test_dataset, batch_size=5, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre = predicted.tolist()
        else:
          y_pre.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names =  ['Nominal', 'Faulty']
print(classification_report(y_0712_transformed.tolist(), y_pre, target_names=target_names))

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 100.0
Iteration:2,  Accuracy: 100.0
Iteration:3,  Accuracy: 100.0
Iteration:4,  Accuracy: 92.0
Iteration:5,  Accuracy: 90.0
Iteration:6,  Accuracy: 88.57142857142857
Iteration:7,  Accuracy: 85.0
Iteration:8,  Accuracy: 84.44444444444444
Iteration:9,  Accuracy: 84.0
Iteration:10,  Accuracy: 85.45454545454545
Iteration:11,  Accuracy: 86.66666666666667
Iteration:12,  Accuracy: 87.6923076923077
Iteration:13,  Accuracy: 88.57142857142857
Iteration:14,  Accuracy: 89.33333333333333
Iteration:15,  Accuracy: 90.0
Iteration:16,  Accuracy: 90.58823529411765
Iteration:17,  Accuracy: 91.11111111111111
Iteration:18,  Accuracy: 91.57894736842105
Iteration:19,  Accuracy: 92.0
Iteration:20,  Accuracy: 92.38095238095238
Iteration:21,  Accuracy: 92.72727272727273
Iteration:22,  Accuracy: 93.04347826086956
Iteration:23,  Accuracy: 92.5
Iteration:24,  Accuracy: 91.2
Iteration:25,  Accuracy: 91.53846153846153
Iteration:26,  Accuracy: 91.85185185185185
It

## Multi class classification
### 5. 21일 데이터 단독 학습
##### 논문 결과: 94%

In [ ]:
print(y_test_0721.shape)
#print(np.bincount(y_train_0721))

(5965,)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 9
batch_size = 120
num_epochs = len(y_train_0721) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_0721, y_train_0721)
train_loader = DataLoader(dataset=train_dataset, batch_size=120, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/198], Step [100/199], Loss: 0.5562757849693298
Epoch [2/198], Step [100/199], Loss: 0.3308072090148926
Epoch [3/198], Step [100/199], Loss: 0.32848304510116577
Epoch [4/198], Step [100/199], Loss: 0.09255971759557724
Epoch [5/198], Step [100/199], Loss: 0.18708154559135437
Epoch [6/198], Step [100/199], Loss: 0.1092614009976387
Epoch [7/198], Step [100/199], Loss: 0.04381103441119194
Epoch [8/198], Step [100/199], Loss: 0.020848872140049934
Epoch [9/198], Step [100/199], Loss: 0.05819854512810707
Epoch [10/198], Step [100/199], Loss: 0.041767846792936325
Epoch [11/198], Step [100/199], Loss: 0.005734832491725683
Epoch [12/198], Step [100/199], Loss: 0.042077790945768356
Epoch [13/198], Step [100/199], Loss: 0.0018070769729092717
Epoch [14/198], Step [100/199], Loss: 0.0005045937723480165
Epoch [15/198], Step [100/199], Loss: 0.0012840890558436513
Epoch [16/198], Step [100/199], Loss: 0.03329910710453987
Epoch [17/198], Step [100/199], Loss: 0.04496325924992561
Epoch [18/198], 

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_0721, y_test_0721)
test_loader = DataLoader(dataset=test_dataset, batch_size=50, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre = predicted.tolist()
        else:
          y_pre.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 'L0.3']
print(classification_report(y_test_0721.tolist(), y_pre, target_names=target_names))

Iteration:0,  Accuracy: 100.0
Iteration:1,  Accuracy: 100.0
Iteration:2,  Accuracy: 100.0
Iteration:3,  Accuracy: 100.0
Iteration:4,  Accuracy: 100.0
Iteration:5,  Accuracy: 100.0
Iteration:6,  Accuracy: 100.0
Iteration:7,  Accuracy: 100.0
Iteration:8,  Accuracy: 100.0
Iteration:9,  Accuracy: 100.0
Iteration:10,  Accuracy: 100.0
Iteration:11,  Accuracy: 100.0
Iteration:12,  Accuracy: 100.0
Iteration:13,  Accuracy: 100.0
Iteration:14,  Accuracy: 100.0
Iteration:15,  Accuracy: 100.0
Iteration:16,  Accuracy: 100.0
Iteration:17,  Accuracy: 100.0
Iteration:18,  Accuracy: 100.0
Iteration:19,  Accuracy: 99.9
Iteration:20,  Accuracy: 99.9047619047619
Iteration:21,  Accuracy: 99.9090909090909
Iteration:22,  Accuracy: 99.91304347826087
Iteration:23,  Accuracy: 99.91666666666667
Iteration:24,  Accuracy: 99.84
Iteration:25,  Accuracy: 99.84615384615384
Iteration:26,  Accuracy: 99.85185185185185
Iteration:27,  Accuracy: 99.85714285714286
Iteration:28,  Accuracy: 99.79310344827586
Iteration:29,  Acc

### 6. 21일 학습 결과로 23일 테스트
##### 논문 결과: 48%

In [9]:
X_train = np.concatenate(( X_train_0721, X_test_0721), axis=0)
y_train = np.concatenate(( y_train_0721, y_test_0721), axis=0)
X_test = np.concatenate(( X_train_0723, X_test_0723), axis=0)
y_test = np.concatenate((y_train_0723, y_test_0723), axis=0)

In [10]:
print(y_train.shape)
print(y_test.shape)

(29821,)
(32267,)


In [11]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 9
batch_size = 160
num_epochs = len(y_train) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=160, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/186], Step [100/187], Loss: 0.5315901041030884
Epoch [2/186], Step [100/187], Loss: 0.20726588368415833
Epoch [3/186], Step [100/187], Loss: 0.10642541944980621
Epoch [4/186], Step [100/187], Loss: 0.03837452828884125
Epoch [5/186], Step [100/187], Loss: 0.03140440583229065
Epoch [6/186], Step [100/187], Loss: 0.04022841900587082
Epoch [7/186], Step [100/187], Loss: 0.012049923650920391
Epoch [8/186], Step [100/187], Loss: 0.03550467640161514
Epoch [9/186], Step [100/187], Loss: 0.002751746214926243
Epoch [10/186], Step [100/187], Loss: 0.025552207604050636
Epoch [11/186], Step [100/187], Loss: 0.09670872986316681
Epoch [12/186], Step [100/187], Loss: 0.0039201779291033745
Epoch [13/186], Step [100/187], Loss: 0.0014876448549330235
Epoch [14/186], Step [100/187], Loss: 0.0016998080536723137
Epoch [15/186], Step [100/187], Loss: 0.061335571110248566
Epoch [16/186], Step [100/187], Loss: 0.044676922261714935
Epoch [17/186], Step [100/187], Loss: 0.00777738681063056
Epoch [18/186

In [12]:
# Test the model
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=190, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        if i==0:
          y_pre = predicted.tolist()
        else:
          y_pre.extend(predicted.tolist())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')

target_names = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 'L0.3']
print(classification_report(y_test.tolist(), y_pre, target_names=target_names))

Iteration:0,  Accuracy: 72.63157894736842
Iteration:1,  Accuracy: 74.47368421052632
Iteration:2,  Accuracy: 74.3859649122807
Iteration:3,  Accuracy: 74.73684210526316
Iteration:4,  Accuracy: 75.15789473684211
Iteration:5,  Accuracy: 75.35087719298245
Iteration:6,  Accuracy: 75.93984962406014
Iteration:7,  Accuracy: 76.05263157894737
Iteration:8,  Accuracy: 76.3157894736842
Iteration:9,  Accuracy: 76.3157894736842
Iteration:10,  Accuracy: 76.45933014354067
Iteration:11,  Accuracy: 76.00877192982456
Iteration:12,  Accuracy: 75.66801619433198
Iteration:13,  Accuracy: 74.81203007518798
Iteration:14,  Accuracy: 74.49122807017544
Iteration:15,  Accuracy: 74.83552631578948
Iteration:16,  Accuracy: 74.6749226006192
Iteration:17,  Accuracy: 74.50292397660819
Iteration:18,  Accuracy: 74.62603878116343
Iteration:19,  Accuracy: 74.65789473684211
Iteration:20,  Accuracy: 74.66165413533835
Iteration:21,  Accuracy: 74.5933014354067
Iteration:22,  Accuracy: 74.62242562929062
Iteration:23,  Accuracy: 7

# GCS data 으로 성능 평가
## 1. Binary Classification

In [13]:
X_train_bin = np.load("X_train_bin.npy")
y_train_bin = np.load("y_train_bin.npy")
y_train_bin = y_train_bin.astype(np.int64)
X_test_bin = np.load("X_test_bin.npy")
y_test_bin = np.load("y_test_bin.npy")
y_test_bin = y_test_bin.astype(np.int64)

In [ ]:
print(y_train_bin.shape)
print(y_test_bin.shape)

(171384,)
(42846,)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = 1000
num_epochs = len(y_train_bin) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_bin, y_train_bin)
train_loader = DataLoader(dataset=train_dataset, batch_size=1000, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/171], Step [100/172], Loss: 0.31622278690338135
Epoch [2/171], Step [100/172], Loss: 0.3009490370750427
Epoch [3/171], Step [100/172], Loss: 0.28477317094802856
Epoch [4/171], Step [100/172], Loss: 0.2578229606151581
Epoch [5/171], Step [100/172], Loss: 0.21452702581882477
Epoch [6/171], Step [100/172], Loss: 0.1627504974603653
Epoch [7/171], Step [100/172], Loss: 0.127942755818367
Epoch [8/171], Step [100/172], Loss: 0.12482455372810364
Epoch [9/171], Step [100/172], Loss: 0.13061602413654327
Epoch [10/171], Step [100/172], Loss: 0.10844194144010544
Epoch [11/171], Step [100/172], Loss: 0.08855195343494415
Epoch [12/171], Step [100/172], Loss: 0.09990593791007996
Epoch [13/171], Step [100/172], Loss: 0.08401764184236526
Epoch [14/171], Step [100/172], Loss: 0.09484050422906876
Epoch [15/171], Step [100/172], Loss: 0.05528608709573746
Epoch [16/171], Step [100/172], Loss: 0.06260576099157333
Epoch [17/171], Step [100/172], Loss: 0.06557295471429825
Epoch [18/171], Step [100/17

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_bin, y_test_bin)
test_loader = DataLoader(dataset=test_dataset, batch_size=400, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 90.5
Iteration:1,  Accuracy: 90.375
Iteration:2,  Accuracy: 89.91666666666667
Iteration:3,  Accuracy: 90.0625
Iteration:4,  Accuracy: 90.25
Iteration:5,  Accuracy: 90.29166666666667
Iteration:6,  Accuracy: 90.17857142857143
Iteration:7,  Accuracy: 90.0
Iteration:8,  Accuracy: 90.16666666666667
Iteration:9,  Accuracy: 90.1
Iteration:10,  Accuracy: 90.11363636363636
Iteration:11,  Accuracy: 90.125
Iteration:12,  Accuracy: 90.0
Iteration:13,  Accuracy: 90.14285714285714
Iteration:14,  Accuracy: 90.18333333333334
Iteration:15,  Accuracy: 90.1875
Iteration:16,  Accuracy: 90.30882352941177
Iteration:17,  Accuracy: 90.25
Iteration:18,  Accuracy: 90.15789473684211
Iteration:19,  Accuracy: 90.175
Iteration:20,  Accuracy: 90.0
Iteration:21,  Accuracy: 89.89772727272727
Iteration:22,  Accuracy: 89.97826086956522
Iteration:23,  Accuracy: 90.0
Iteration:24,  Accuracy: 89.99
Iteration:25,  Accuracy: 90.0576923076923
Iteration:26,  Accuracy: 90.01851851851852
Iteration:27,  Ac

## 2. 3-class multi Classification

In [ ]:
X_train_three = np.load("X_train_three.npy")
y_train_three = np.load("y_train_three.npy")
y_train_three = y_train_three.astype(np.int64)
X_test_three = np.load("X_test_three.npy")
y_test_three = np.load("y_test_three.npy")
y_test_three = y_test_three.astype(np.int64)

In [ ]:
print(y_train_three.shape)
print(y_test_three.shape)

(50192,)
(12548,)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 3
batch_size = 270
num_epochs = len(y_train_three) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_three, y_train_three)
train_loader = DataLoader(dataset=train_dataset, batch_size=270, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/185], Step [100/186], Loss: 0.3426899313926697
Epoch [2/185], Step [100/186], Loss: 0.3482637107372284
Epoch [3/185], Step [100/186], Loss: 0.3827095627784729
Epoch [4/185], Step [100/186], Loss: 0.22269335389137268
Epoch [5/185], Step [100/186], Loss: 0.21475282311439514
Epoch [6/185], Step [100/186], Loss: 0.21794582903385162
Epoch [7/185], Step [100/186], Loss: 0.17095205187797546
Epoch [8/185], Step [100/186], Loss: 0.13830289244651794
Epoch [9/185], Step [100/186], Loss: 0.14321444928646088
Epoch [10/185], Step [100/186], Loss: 0.16504447162151337
Epoch [11/185], Step [100/186], Loss: 0.09946553409099579
Epoch [12/185], Step [100/186], Loss: 0.09435024112462997
Epoch [13/185], Step [100/186], Loss: 0.11817925423383713
Epoch [14/185], Step [100/186], Loss: 0.07228156179189682
Epoch [15/185], Step [100/186], Loss: 0.06672212481498718
Epoch [16/185], Step [100/186], Loss: 0.0883413627743721
Epoch [17/185], Step [100/186], Loss: 0.08756371587514877
Epoch [18/185], Step [100/1

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_three, y_test_three)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 88.0
Iteration:1,  Accuracy: 87.0
Iteration:2,  Accuracy: 89.33333333333333
Iteration:3,  Accuracy: 90.0
Iteration:4,  Accuracy: 90.4
Iteration:5,  Accuracy: 89.5
Iteration:6,  Accuracy: 88.71428571428571
Iteration:7,  Accuracy: 88.5
Iteration:8,  Accuracy: 88.77777777777777
Iteration:9,  Accuracy: 88.4
Iteration:10,  Accuracy: 88.81818181818181
Iteration:11,  Accuracy: 89.33333333333333
Iteration:12,  Accuracy: 89.23076923076923
Iteration:13,  Accuracy: 89.21428571428571
Iteration:14,  Accuracy: 89.46666666666667
Iteration:15,  Accuracy: 89.5
Iteration:16,  Accuracy: 89.76470588235294
Iteration:17,  Accuracy: 89.55555555555556
Iteration:18,  Accuracy: 89.47368421052632
Iteration:19,  Accuracy: 89.4
Iteration:20,  Accuracy: 89.38095238095238
Iteration:21,  Accuracy: 89.54545454545455
Iteration:22,  Accuracy: 89.30434782608695
Iteration:23,  Accuracy: 89.45833333333333
Iteration:24,  Accuracy: 89.6
Iteration:25,  Accuracy: 89.61538461538461
Iteration:26,  Accurac

## 3. 9-class multi Classification

In [ ]:
X_train_nine = np.load("X_train_nine.npy")
y_train_nine = np.load("y_train_nine.npy")
y_train_nine = y_train_nine.astype(np.int64)
X_test_nine = np.load("X_test_nine.npy")
y_test_nine = np.load("y_test_nine.npy")
y_test_nine = y_test_nine.astype(np.int64)

In [ ]:
print(y_train_nine.shape)
print(y_test_nine.shape)

(23696,)
(5924,)


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
sequence_length = 20
input_size = 8
hidden_size = 64
num_layers = 2
num_classes = 9
batch_size = 160
num_epochs = len(y_train_nine) // batch_size # 2
learning_rate = 0.01

class CustomDataset(Dataset):
  def __init__(self, X_data, Y_data):
    self.x_data = X_data
    self.y_data = Y_data

  # 총 데이터의 개수를 리턴
  def __len__(self):
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = self.y_data[idx]  # y_data가 list 안에 4900 여개의 숫자가 들어있는 식의 형태...? 이다보니 구조 변경이 필요함. 이렇게 하면 그냥 숫자 하나 받아지는 것임
    # 내지는
    return x, y


# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = CustomDataset(X_train_nine, y_train_nine)
train_loader = DataLoader(dataset=train_dataset, batch_size=160, shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/148], Step [100/149], Loss: 1.1275221109390259
Epoch [2/148], Step [100/149], Loss: 0.9590297937393188
Epoch [3/148], Step [100/149], Loss: 0.9057259559631348
Epoch [4/148], Step [100/149], Loss: 0.8030657768249512
Epoch [5/148], Step [100/149], Loss: 0.8367018699645996
Epoch [6/148], Step [100/149], Loss: 0.6803274154663086
Epoch [7/148], Step [100/149], Loss: 0.6746034622192383
Epoch [8/148], Step [100/149], Loss: 0.43692389130592346
Epoch [9/148], Step [100/149], Loss: 0.6171703934669495
Epoch [10/148], Step [100/149], Loss: 0.3945068120956421
Epoch [11/148], Step [100/149], Loss: 0.2965227961540222
Epoch [12/148], Step [100/149], Loss: 0.22448770701885223
Epoch [13/148], Step [100/149], Loss: 0.3887699544429779
Epoch [14/148], Step [100/149], Loss: 0.21288970112800598
Epoch [15/148], Step [100/149], Loss: 0.1538824737071991
Epoch [16/148], Step [100/149], Loss: 0.3264177441596985
Epoch [17/148], Step [100/149], Loss: 0.18274766206741333
Epoch [18/148], Step [100/149], Loss

In [ ]:
# Test the model
test_dataset = CustomDataset(X_test_nine, y_test_nine)
test_loader = DataLoader(dataset=test_dataset, batch_size=60, shuffle=False)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Iteration:{},  Accuracy: {}'.format(i, 100 * correct / total))

    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Iteration:0,  Accuracy: 91.66666666666667
Iteration:1,  Accuracy: 91.66666666666667
Iteration:2,  Accuracy: 92.77777777777777
Iteration:3,  Accuracy: 92.5
Iteration:4,  Accuracy: 92.0
Iteration:5,  Accuracy: 92.5
Iteration:6,  Accuracy: 93.57142857142857
Iteration:7,  Accuracy: 93.75
Iteration:8,  Accuracy: 93.88888888888889
Iteration:9,  Accuracy: 94.33333333333333
Iteration:10,  Accuracy: 94.39393939393939
Iteration:11,  Accuracy: 93.88888888888889
Iteration:12,  Accuracy: 93.71794871794872
Iteration:13,  Accuracy: 93.80952380952381
Iteration:14,  Accuracy: 93.66666666666667
Iteration:15,  Accuracy: 93.64583333333333
Iteration:16,  Accuracy: 93.62745098039215
Iteration:17,  Accuracy: 93.51851851851852
Iteration:18,  Accuracy: 93.42105263157895
Iteration:19,  Accuracy: 93.0
Iteration:20,  Accuracy: 93.01587301587301
Iteration:21,  Accuracy: 92.72727272727273
Iteration:22,  Accuracy: 92.68115942028986
Iteration:23,  Accuracy: 92.98611111111111
Iteration:24,  Accuracy: 93.0
Iteration:25